# Full text search and keyword association analysis

全文檢索與關聯分析

# Load preprocessed news dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('preprocessed_news_218posts.csv',sep='|')

In [3]:
df.head(1)

,item_id,date,title,content,sentiment,summary,top_key_freq,tokens,tokens_rm_stopwords,entities,token_pos,link,photo_link,category
0,aipl_20220929_1,2022-09-29,台灣國際漁業展、智慧農業週開幕 逾15國參展,外交部政次田中光今天出席2022年「台灣國際漁業展」與「台灣智慧農業週」聯合開幕儀式，他表示...,0.68,"['外交部政次田中光今天出席2022年「台灣國際漁業展」與「台灣智慧農業週」聯合開幕儀式',...","[('展覽', 8), ('台灣', 6), ('廠商', 6), ('參展', 6), (...","['外交部', '政次', '田中光', '今天', '出席', '2022年', '「',...","['外交部', '政次', '田中光', '出席', '台灣', '國際', '漁業展', ...","{(657, 658, 'GPE', '台'), (143, 146, 'PERSON', ...","[('外交部', 'Nc'), ('政次', 'Na'), ('田中光', 'Nb'), (...",https://www.cna.com.tw/news/aipl/202209290375....,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...,政治


# Filter data by searching keywords from "content" column (全文檢索)

In [4]:
queryKey = '智慧農業'
df_query = df[df['content'].str.contains(queryKey)]

In [5]:
df_query.shape

(2, 14)

# Filter data using the following function

In [8]:
from datetime import datetime, timedelta

In [9]:
# This function filter data by searching keywords from "content" column. We have already used it in the previous App.
def filter_dataFrame_fullText(user_keywords, cond, cate,weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')

    # proceed filtering
    if (cate == "全部") & (cond == 'and'):
        df_query = df[(df.date >= start_date) & (df.date <= end_date) & df.content.apply(
            lambda row: all((qk in row) for qk in user_keywords))]
    elif (cate == "全部") & (cond == 'or'):
        queryKey = '|'.join(user_keywords)
        df_query = df[(df['date'] >= start_date) & (df['date'] <= end_date) & df.content.str.contains(queryKey)]
    elif (cond == 'and'):
        df_query = df[(df.category == cate) & (df.date >= start_date) & (df.date <= end_date) & df.content.apply(
            lambda row: all((qk in row) for qk in user_keywords))]
    elif (cond == 'or'):
        queryKey = '|'.join(user_keywords)
        df_query = df[(df.category == cate) & (df['date'] >= start_date) & (df['date'] <= end_date) & df[
            'content'].str.contains(queryKey)]

    return df_query

In [10]:
key = ['農業']
cond = 'and'
cate = '全部'
weeks = 2

In [11]:
df_query = filter_dataFrame_fullText(key, cond, cate,weeks)

In [12]:
df_query.shape

(3, 14)

# Get news title, category, and link

In [13]:
key = ['智慧','農業']
cond = 'and'
cate = '全部'
weeks = 2
df_query = filter_dataFrame_fullText(queryKey, cond, cate, weeks)

In [15]:
len(df_query)

2

In [18]:
for i in range(len(df_query)):   
    print(i)
    print(df_query.iloc[i]['category'])
    print(df_query.iloc[i]['title'])
    print(df_query.iloc[i].link)
    print(df_query.iloc[i].photo_link)

0
政治
台灣國際漁業展、智慧農業週開幕 逾15國參展
https://www.cna.com.tw/news/aipl/202209290375.aspx
https://imgcdn.cna.com.tw/www/WebPhotos/200/20220929/1024x768_20220929000213.jpg
1
政治
韓駐台代表：台韓半導體業緊密相連使彼此獲益
https://www.cna.com.tw/news/aipl/202209290365.aspx
https://imgcdn.cna.com.tw/www/WebPhotos/200/20220929/977x768_20220929000216.jpg


In [19]:
df_query.iloc[0]['photo_link']

'https://imgcdn.cna.com.tw/www/WebPhotos/200/20220929/1024x768_20220929000213.jpg'

## If the photo_link value is 'nan', it will be a problem when converted to json format on Django!

### Approach 1: test the photo_link is NaN or not and replace it with empty string

In [21]:
import pandas as pd
import numpy as np
import math

# we can use pandas, numpy or math to check if photo_link is NaN
x = float("nan")

print(f"It's pd.isna  : {pd.isna(x)}")
print(f"It's np.isnan  : {np.isnan(x)}")
print(f"It's math.isnan : {math.isnan(x)}")

It's pd.isna  : True
It's np.isnan  : True
It's math.isnan : True


In [22]:
df_query.iloc[0]['photo_link']

'https://imgcdn.cna.com.tw/www/WebPhotos/200/20220929/1024x768_20220929000213.jpg'

In [23]:
photo_link = df_query.iloc[0]['photo_link']
# if photo_link value is NaN, replace it with empty string 
if pd.isna(photo_link):
    photo_link=''

In [24]:
photo_link

'https://imgcdn.cna.com.tw/www/WebPhotos/200/20220929/1024x768_20220929000213.jpg'

### Approach 2: Replace nan with None

## All-in-one function to return title, link, and category

In [25]:
key = ['智慧','農業']
cond = 'and'
cate = '全部'
weeks = 2
df_query = filter_dataFrame_fullText(queryKey, cond, cate, weeks)

In [26]:

# get titles and links from k pieces of news 
def get_title_link_topk(df_query, k = 25):
    items = []
    for i in range( len(df_query[0:k]) ): # show only 10 news
        category = df_query.iloc[i]['category']
        title = df_query.iloc[i]['title']
        link = df_query.iloc[i]['link']
        photo_link = df_query.iloc[i]['photo_link']

        # if photo_link value is NaN, replace it with empty string 
        if pd.isna(photo_link):
            photo_link=''
        
        item_info = {
            'category': category, 
            'title': title, 
            'link': link, 
            'photo_link': photo_link
        }

        items.append(item_info)
    return items 


In [27]:
get_title_link_topk(df_query, 5)

[{'category': '政治',
  'title': '台灣國際漁業展、智慧農業週開幕 逾15國參展',
  'link': 'https://www.cna.com.tw/news/aipl/202209290375.aspx',
  'photo_link': 'https://imgcdn.cna.com.tw/www/WebPhotos/200/20220929/1024x768_20220929000213.jpg'},
 {'category': '政治',
  'title': '韓駐台代表：台韓半導體業緊密相連使彼此獲益',
  'link': 'https://www.cna.com.tw/news/aipl/202209290365.aspx',
  'photo_link': 'https://imgcdn.cna.com.tw/www/WebPhotos/200/20220929/977x768_20220929000216.jpg'}]

# Find words relating to the keywords

## Find related words from the top_key_freq column

相關詞有哪一些? 找出各篇文章的topk關鍵詞?

In [28]:
df_query.top_key_freq

0    [('展覽', 8), ('台灣', 6), ('廠商', 6), ('參展', 6), (...
3    [('韓國', 11), ('台灣', 8), ('鄭炳元', 7), ('產業', 5),...
Name: top_key_freq, dtype: object

### From the word_freq pairs, we sum frequency for each word

In [27]:
# What is this line?
# if w in all_pairs:

In [31]:
df_query.iloc[0].top_key_freq

"[('展覽', 8), ('台灣', 6), ('廠商', 6), ('參展', 6), ('貿易', 5), ('代表', 5), ('田中光', 4), ('國際', 4), ('規模', 3), ('國家', 3), ('訂閱', 3), ('大使館', 3), ('出席', 2), ('漁業展', 2), ('智慧', 2), ('國內外', 2), ('大使', 2), ('聖文森', 2), ('格瑞那丁', 2), ('印尼', 2), ('經濟', 2), ('主任', 2), ('匈牙利', 2), ('辦事處', 2), ('商務', 2), ('國際性', 2), ('漁業', 2), ('世界', 2), ('經貿', 2), ('外交部', 1), ('政次', 1), ('農業', 1), ('儀式', 1), ('使節', 1), ('馬紹爾', 1), ('群島', 1), ('艾芮瓊', 1), ('柏安卓', 1), ('德經', 1), ('辦事處長', 1), ('林百科', 1), ('馬來西亞', 1), ('友誼', 1), ('中心', 1), ('何瑞萍', 1), ('台北', 1), ('蘇迪佑', 1), ('投資', 1), ('戴博斯', 1), ('漁產', 1), ('業界', 1), ('嘉年華', 1), ('產學', 1), ('機構', 1), ('產品', 1), ('技術', 1), ('電子報', 1), ('掌握', 1), ('天下', 1), ('輸入', 1), ('電子', 1), ('信箱', 1), ('格式', 1), ('吐瓦魯國', 1), ('巴布亞', 1), ('紐幾內亞', 1), ('史瓦帝尼', 1), ('王國', 1), ('攤位', 1), ('外交團', 1), ('異國', 1), ('風采', 1), ('COVID-19', 1), ('疫情', 1), ('衝擊', 1), ('實體', 1), ('台灣版', 1), ('泡泡', 1), ('經營', 1), ('田中光韓', 1), ('半導體業', 1), ('獲益', 1), ('舉辦', 1), ('酒會', 1), ('關係', 1)]"

In [32]:
eval(df_query.iloc[0].top_key_freq)

[('展覽', 8),
 ('台灣', 6),
 ('廠商', 6),
 ('參展', 6),
 ('貿易', 5),
 ('代表', 5),
 ('田中光', 4),
 ('國際', 4),
 ('規模', 3),
 ('國家', 3),
 ('訂閱', 3),
 ('大使館', 3),
 ('出席', 2),
 ('漁業展', 2),
 ('智慧', 2),
 ('國內外', 2),
 ('大使', 2),
 ('聖文森', 2),
 ('格瑞那丁', 2),
 ('印尼', 2),
 ('經濟', 2),
 ('主任', 2),
 ('匈牙利', 2),
 ('辦事處', 2),
 ('商務', 2),
 ('國際性', 2),
 ('漁業', 2),
 ('世界', 2),
 ('經貿', 2),
 ('外交部', 1),
 ('政次', 1),
 ('農業', 1),
 ('儀式', 1),
 ('使節', 1),
 ('馬紹爾', 1),
 ('群島', 1),
 ('艾芮瓊', 1),
 ('柏安卓', 1),
 ('德經', 1),
 ('辦事處長', 1),
 ('林百科', 1),
 ('馬來西亞', 1),
 ('友誼', 1),
 ('中心', 1),
 ('何瑞萍', 1),
 ('台北', 1),
 ('蘇迪佑', 1),
 ('投資', 1),
 ('戴博斯', 1),
 ('漁產', 1),
 ('業界', 1),
 ('嘉年華', 1),
 ('產學', 1),
 ('機構', 1),
 ('產品', 1),
 ('技術', 1),
 ('電子報', 1),
 ('掌握', 1),
 ('天下', 1),
 ('輸入', 1),
 ('電子', 1),
 ('信箱', 1),
 ('格式', 1),
 ('吐瓦魯國', 1),
 ('巴布亞', 1),
 ('紐幾內亞', 1),
 ('史瓦帝尼', 1),
 ('王國', 1),
 ('攤位', 1),
 ('外交團', 1),
 ('異國', 1),
 ('風采', 1),
 ('COVID-19', 1),
 ('疫情', 1),
 ('衝擊', 1),
 ('實體', 1),
 ('台灣版', 1),
 ('泡泡', 1),
 ('經營', 1),
 ('田中光韓', 1),

In [33]:
all_pairs = {}

In [34]:
all_pairs

{}

In [35]:
# Process the first piece of news
row = df_query.iloc[0].top_key_freq
pairs = eval(row)
for pair in pairs:
    w,f = pair
    if w in all_pairs:
        all_pairs[w]+= f
    else:
        all_pairs[w] = f

In [36]:
all_pairs

{'展覽': 8,
 '台灣': 6,
 '廠商': 6,
 '參展': 6,
 '貿易': 5,
 '代表': 5,
 '田中光': 4,
 '國際': 4,
 '規模': 3,
 '國家': 3,
 '訂閱': 3,
 '大使館': 3,
 '出席': 2,
 '漁業展': 2,
 '智慧': 2,
 '國內外': 2,
 '大使': 2,
 '聖文森': 2,
 '格瑞那丁': 2,
 '印尼': 2,
 '經濟': 2,
 '主任': 2,
 '匈牙利': 2,
 '辦事處': 2,
 '商務': 2,
 '國際性': 2,
 '漁業': 2,
 '世界': 2,
 '經貿': 2,
 '外交部': 1,
 '政次': 1,
 '農業': 1,
 '儀式': 1,
 '使節': 1,
 '馬紹爾': 1,
 '群島': 1,
 '艾芮瓊': 1,
 '柏安卓': 1,
 '德經': 1,
 '辦事處長': 1,
 '林百科': 1,
 '馬來西亞': 1,
 '友誼': 1,
 '中心': 1,
 '何瑞萍': 1,
 '台北': 1,
 '蘇迪佑': 1,
 '投資': 1,
 '戴博斯': 1,
 '漁產': 1,
 '業界': 1,
 '嘉年華': 1,
 '產學': 1,
 '機構': 1,
 '產品': 1,
 '技術': 1,
 '電子報': 1,
 '掌握': 1,
 '天下': 1,
 '輸入': 1,
 '電子': 1,
 '信箱': 1,
 '格式': 1,
 '吐瓦魯國': 1,
 '巴布亞': 1,
 '紐幾內亞': 1,
 '史瓦帝尼': 1,
 '王國': 1,
 '攤位': 1,
 '外交團': 1,
 '異國': 1,
 '風采': 1,
 'COVID-19': 1,
 '疫情': 1,
 '衝擊': 1,
 '實體': 1,
 '台灣版': 1,
 '泡泡': 1,
 '經營': 1,
 '田中光韓': 1,
 '半導體業': 1,
 '獲益': 1,
 '舉辦': 1,
 '酒會': 1,
 '關係': 1}

In [37]:
# Process the second piece of news
row = df_query.iloc[1].top_key_freq
pairs = eval(row)
for pair in pairs:
    w,f = pair
    if w in all_pairs:
        all_pairs[w]+= f
    else:
        all_pairs[w] = f

In [38]:
all_pairs

{'展覽': 8,
 '台灣': 14,
 '廠商': 6,
 '參展': 7,
 '貿易': 8,
 '代表': 9,
 '田中光': 7,
 '國際': 6,
 '規模': 4,
 '國家': 5,
 '訂閱': 6,
 '大使館': 3,
 '出席': 4,
 '漁業展': 3,
 '智慧': 3,
 '國內外': 2,
 '大使': 2,
 '聖文森': 2,
 '格瑞那丁': 2,
 '印尼': 3,
 '經濟': 2,
 '主任': 2,
 '匈牙利': 2,
 '辦事處': 2,
 '商務': 2,
 '國際性': 2,
 '漁業': 2,
 '世界': 3,
 '經貿': 4,
 '外交部': 2,
 '政次': 1,
 '農業': 2,
 '儀式': 1,
 '使節': 2,
 '馬紹爾': 1,
 '群島': 1,
 '艾芮瓊': 1,
 '柏安卓': 1,
 '德經': 1,
 '辦事處長': 1,
 '林百科': 1,
 '馬來西亞': 1,
 '友誼': 1,
 '中心': 1,
 '何瑞萍': 1,
 '台北': 4,
 '蘇迪佑': 1,
 '投資': 1,
 '戴博斯': 1,
 '漁產': 1,
 '業界': 1,
 '嘉年華': 1,
 '產學': 1,
 '機構': 1,
 '產品': 1,
 '技術': 1,
 '電子報': 2,
 '掌握': 2,
 '天下': 2,
 '輸入': 2,
 '電子': 2,
 '信箱': 2,
 '格式': 2,
 '吐瓦魯國': 1,
 '巴布亞': 1,
 '紐幾內亞': 1,
 '史瓦帝尼': 1,
 '王國': 1,
 '攤位': 1,
 '外交團': 1,
 '異國': 1,
 '風采': 1,
 'COVID-19': 1,
 '疫情': 1,
 '衝擊': 1,
 '實體': 1,
 '台灣版': 1,
 '泡泡': 1,
 '經營': 1,
 '田中光韓': 1,
 '半導體業': 2,
 '獲益': 3,
 '舉辦': 3,
 '酒會': 4,
 '關係': 3,
 '韓國': 11,
 '鄭炳元': 7,
 '產業': 5,
 '出口': 4,
 '致詞': 3,
 '半導體': 3,
 '區域': 3,
 '會議': 3,
 '互補性': 2,
 '代表部': 2,
 

In [39]:
from collections import Counter

In [40]:
# Then we can get the most frequent words using Counter

counter = Counter(all_pairs)

In [41]:
counter.most_common(5)

[('台灣', 14), ('韓國', 11), ('代表', 9), ('展覽', 8), ('貿易', 8)]

## All-in-one function: Get related keywords

In [42]:
key = ['智慧']
cond = 'and'
cate = '全部'
weeks = 2

In [43]:
df_query = filter_dataFrame_fullText(key, cond, cate,weeks)

In [44]:
df_query.shape

(11, 14)

In [45]:
from collections import Counter

# 相關詞有哪一些?找出各篇文章的topk關鍵詞加以彙整計算
# 不能用 "get_related_keys"當函數名稱，因為這是Django系統用的名稱
def get_related_words(df_query, key):
    all_pairs={}
    for idx in range(len(df_query)):
        row = df_query.iloc[idx].top_key_freq
        pairs = eval(row)
        for pair in pairs:
            w,f = pair
            if w in all_pairs:
                all_pairs[w]+= f
            else:
                all_pairs[w] = f

    counter = Counter(all_pairs)
    return counter.most_common(20) #return list format
    #return dict(counter.most_common(20)) #return dict format

In [46]:
get_related_words(df_query, '智慧')

[('台灣', 55),
 ('訂閱', 24),
 ('國際', 21),
 ('林智堅', 19),
 ('國家', 16),
 ('智慧', 15),
 ('代表', 13),
 ('論文', 13),
 ('台北', 12),
 ('技術', 12),
 ('產業', 12),
 ('韓國', 11),
 ('文化', 11),
 ('美國', 11),
 ('數據', 11),
 ('關鍵', 11),
 ('貿易', 10),
 ('世界', 10),
 ('醫療', 10),
 ('發表', 10)]

## Process wordcloud data

In [47]:
# Get related keywords by counting the top keywords of each news.
# Notice:  do not name function as  "get_related_keys",
# because this name is used in Django
def get_related_words(df_query, key):

    all_pairs = {}
    for idx in range(len(df_query)):
        row = df_query.iloc[idx].top_key_freq # column name may be top_keys_freq
        pairs = eval(row)

        # Get related keywords by counting the top keywords of each news.
        for pair in pairs:
            w, f = pair
            if w in all_pairs:
                all_pairs[w] += f
            else:
                all_pairs[w] = f
    counter = Counter(all_pairs)

    wf_pairs = counter.most_common(30)

    # cloud chart data
    # the minimum and maximum frequency of top words
    min_ = wf_pairs[-1][1]  # the last line is smaller
    max_ = wf_pairs[0][1]
    # text size based on the value of word frequency for drawing cloud chart
    textSizeMin = 20
    textSizeMax = 120
    # Scaling frequency value into an interval of from 20 to 120.
    clouddata = [{'text': w, 'size': int(textSizeMin + (f - min_) / (max_ - min_) * (textSizeMax - textSizeMin))}
                 for w, f in wf_pairs]

    return   wf_pairs, clouddata 

In [53]:
get_related_words(df_query, '台灣')

([('台灣', 55),
  ('訂閱', 24),
  ('國際', 21),
  ('林智堅', 19),
  ('國家', 16),
  ('智慧', 15),
  ('代表', 13),
  ('論文', 13),
  ('台北', 12),
  ('技術', 12),
  ('產業', 12),
  ('韓國', 11),
  ('文化', 11),
  ('美國', 11),
  ('數據', 11),
  ('關鍵', 11),
  ('貿易', 10),
  ('世界', 10),
  ('醫療', 10),
  ('發表', 10),
  ('國科會', 10),
  ('余正煌', 10),
  ('電子', 9),
  ('舉辦', 9),
  ('晶片', 9),
  ('搞笑', 9),
  ('研究', 9),
  ('遠通', 9),
  ('碩士', 9),
  ('展覽', 8)],
 [{'text': '台灣', 'size': 120},
  {'text': '訂閱', 'size': 54},
  {'text': '國際', 'size': 47},
  {'text': '林智堅', 'size': 43},
  {'text': '國家', 'size': 37},
  {'text': '智慧', 'size': 34},
  {'text': '代表', 'size': 30},
  {'text': '論文', 'size': 30},
  {'text': '台北', 'size': 28},
  {'text': '技術', 'size': 28},
  {'text': '產業', 'size': 28},
  {'text': '韓國', 'size': 26},
  {'text': '文化', 'size': 26},
  {'text': '美國', 'size': 26},
  {'text': '數據', 'size': 26},
  {'text': '關鍵', 'size': 26},
  {'text': '貿易', 'size': 24},
  {'text': '世界', 'size': 24},
  {'text': '醫療', 'size': 24},
  {'text': '

# Find paragraphs containing the keywords. 

There may be too many related paragraphs, so we display only some of them on our Django website

In [62]:
import re

In [61]:
text = df_query.iloc[0].content

In [59]:
text

'外交部政次田中光今天出席2022年「台灣國際漁業展」與「台灣智慧農業週」聯合開幕儀式，他表示，今年有超過15國約270家廠商參展，展覽規模、參展國家與國內外廠商均逐年增加。多位駐台使節今天出席展覽，包括馬紹爾群島大使艾芮瓊（Neijon Rema Edwards）、聖文森及格瑞那丁大使柏安卓（Andrea Clare Bowman）、德經辦事處長林百科（Axel Limberg）、馬來西亞友誼及貿易中心代表何瑞萍（Sharon Ho Swee Peng）、駐台北印尼經濟貿易代表處貿易主任蘇迪佑（Sulistyono Tarno）、匈牙利貿易辦事處商務投資主任戴博斯（Balazs Szabo）等。田中光表示，台灣智慧農業週暨台灣國際漁業展是台灣農、漁產業界的國際性嘉年華展覽，已邁向第8年，展覽規模、參展的國家與國內外廠商均逐年增加，感謝參展廠商及產學研究機構的努力，將台灣農、漁業優質產品及技術在國際展覽中呈現。訂閱《早安世界》電子報 每天3分鐘掌握10件天下事 請輸入正確的電子信箱格式訂閱感謝您的訂閱！田中光指出，今年有超過15個國家、大約270家廠商參展，其中包括吐瓦魯國大使館、巴布亞紐幾內亞駐台商務代表處、聖文森及格瑞那丁大使館、史瓦帝尼王國大使館、匈牙利貿易辦事處的參展攤位，感謝國際廠商以及外交團的參與，使展覽更具國際性及異國風采。田中光表示，在世界經濟受到COVID-19疫情嚴重衝擊下，這項實體展覽，就像是台灣版的「農、漁業經貿泡泡」，期盼透過大家努力經營，使展覽規模更形壯大。田中光韓駐台代表：台韓半導體業緊密相連使彼此獲益印尼舉辦國慶酒會 駐台代表盼深化與台經貿關係'

In [63]:
re.split(r'[。！!？?]', text) 

['外交部政次田中光今天出席2022年「台灣國際漁業展」與「台灣智慧農業週」聯合開幕儀式，他表示，今年有超過15國約270家廠商參展，展覽規模、參展國家與國內外廠商均逐年增加',
 '多位駐台使節今天出席展覽，包括馬紹爾群島大使艾芮瓊（Neijon Rema Edwards）、聖文森及格瑞那丁大使柏安卓（Andrea Clare Bowman）、德經辦事處長林百科（Axel Limberg）、馬來西亞友誼及貿易中心代表何瑞萍（Sharon Ho Swee Peng）、駐台北印尼經濟貿易代表處貿易主任蘇迪佑（Sulistyono Tarno）、匈牙利貿易辦事處商務投資主任戴博斯（Balazs Szabo）等',
 '田中光表示，台灣智慧農業週暨台灣國際漁業展是台灣農、漁產業界的國際性嘉年華展覽，已邁向第8年，展覽規模、參展的國家與國內外廠商均逐年增加，感謝參展廠商及產學研究機構的努力，將台灣農、漁業優質產品及技術在國際展覽中呈現',
 '訂閱《早安世界》電子報 每天3分鐘掌握10件天下事 請輸入正確的電子信箱格式訂閱感謝您的訂閱',
 '田中光指出，今年有超過15個國家、大約270家廠商參展，其中包括吐瓦魯國大使館、巴布亞紐幾內亞駐台商務代表處、聖文森及格瑞那丁大使館、史瓦帝尼王國大使館、匈牙利貿易辦事處的參展攤位，感謝國際廠商以及外交團的參與，使展覽更具國際性及異國風采',
 '田中光表示，在世界經濟受到COVID-19疫情嚴重衝擊下，這項實體展覽，就像是台灣版的「農、漁業經貿泡泡」，期盼透過大家努力經營，使展覽規模更形壯大',
 '田中光韓駐台代表：台韓半導體業緊密相連使彼此獲益印尼舉辦國慶酒會 駐台代表盼深化與台經貿關係']

In [64]:
# the same result
re.split(r'[。|！|!|？|?]', text) 

['外交部政次田中光今天出席2022年「台灣國際漁業展」與「台灣智慧農業週」聯合開幕儀式，他表示，今年有超過15國約270家廠商參展，展覽規模、參展國家與國內外廠商均逐年增加',
 '多位駐台使節今天出席展覽，包括馬紹爾群島大使艾芮瓊（Neijon Rema Edwards）、聖文森及格瑞那丁大使柏安卓（Andrea Clare Bowman）、德經辦事處長林百科（Axel Limberg）、馬來西亞友誼及貿易中心代表何瑞萍（Sharon Ho Swee Peng）、駐台北印尼經濟貿易代表處貿易主任蘇迪佑（Sulistyono Tarno）、匈牙利貿易辦事處商務投資主任戴博斯（Balazs Szabo）等',
 '田中光表示，台灣智慧農業週暨台灣國際漁業展是台灣農、漁產業界的國際性嘉年華展覽，已邁向第8年，展覽規模、參展的國家與國內外廠商均逐年增加，感謝參展廠商及產學研究機構的努力，將台灣農、漁業優質產品及技術在國際展覽中呈現',
 '訂閱《早安世界》電子報 每天3分鐘掌握10件天下事 請輸入正確的電子信箱格式訂閱感謝您的訂閱',
 '田中光指出，今年有超過15個國家、大約270家廠商參展，其中包括吐瓦魯國大使館、巴布亞紐幾內亞駐台商務代表處、聖文森及格瑞那丁大使館、史瓦帝尼王國大使館、匈牙利貿易辦事處的參展攤位，感謝國際廠商以及外交團的參與，使展覽更具國際性及異國風采',
 '田中光表示，在世界經濟受到COVID-19疫情嚴重衝擊下，這項實體展覽，就像是台灣版的「農、漁業經貿泡泡」，期盼透過大家努力經營，使展覽規模更形壯大',
 '田中光韓駐台代表：台韓半導體業緊密相連使彼此獲益印尼舉辦國慶酒會 駐台代表盼深化與台經貿關係']

In [65]:
# the same result: using a backslash in front of ? and !
re.split(r'[。|！|\!|？|\?]', text) 

['外交部政次田中光今天出席2022年「台灣國際漁業展」與「台灣智慧農業週」聯合開幕儀式，他表示，今年有超過15國約270家廠商參展，展覽規模、參展國家與國內外廠商均逐年增加',
 '多位駐台使節今天出席展覽，包括馬紹爾群島大使艾芮瓊（Neijon Rema Edwards）、聖文森及格瑞那丁大使柏安卓（Andrea Clare Bowman）、德經辦事處長林百科（Axel Limberg）、馬來西亞友誼及貿易中心代表何瑞萍（Sharon Ho Swee Peng）、駐台北印尼經濟貿易代表處貿易主任蘇迪佑（Sulistyono Tarno）、匈牙利貿易辦事處商務投資主任戴博斯（Balazs Szabo）等',
 '田中光表示，台灣智慧農業週暨台灣國際漁業展是台灣農、漁產業界的國際性嘉年華展覽，已邁向第8年，展覽規模、參展的國家與國內外廠商均逐年增加，感謝參展廠商及產學研究機構的努力，將台灣農、漁業優質產品及技術在國際展覽中呈現',
 '訂閱《早安世界》電子報 每天3分鐘掌握10件天下事 請輸入正確的電子信箱格式訂閱感謝您的訂閱',
 '田中光指出，今年有超過15個國家、大約270家廠商參展，其中包括吐瓦魯國大使館、巴布亞紐幾內亞駐台商務代表處、聖文森及格瑞那丁大使館、史瓦帝尼王國大使館、匈牙利貿易辦事處的參展攤位，感謝國際廠商以及外交團的參與，使展覽更具國際性及異國風采',
 '田中光表示，在世界經濟受到COVID-19疫情嚴重衝擊下，這項實體展覽，就像是台灣版的「農、漁業經貿泡泡」，期盼透過大家努力經營，使展覽規模更形壯大',
 '田中光韓駐台代表：台韓半導體業緊密相連使彼此獲益印尼舉辦國慶酒會 駐台代表盼深化與台經貿關係']

## Define cut_paragraph() function

In [66]:
# split paragraghes in text
def cut_paragraph(text):
    result = re.split(r'[。|！|!|？|?]', text)
    result = list(filter(None, result))
    return result

In [67]:
paragraphs = cut_paragraph(text)

In [68]:
paragraphs

['外交部政次田中光今天出席2022年「台灣國際漁業展」與「台灣智慧農業週」聯合開幕儀式，他表示，今年有超過15國約270家廠商參展，展覽規模、參展國家與國內外廠商均逐年增加',
 '多位駐台使節今天出席展覽，包括馬紹爾群島大使艾芮瓊（Neijon Rema Edwards）、聖文森及格瑞那丁大使柏安卓（Andrea Clare Bowman）、德經辦事處長林百科（Axel Limberg）、馬來西亞友誼及貿易中心代表何瑞萍（Sharon Ho Swee Peng）、駐台北印尼經濟貿易代表處貿易主任蘇迪佑（Sulistyono Tarno）、匈牙利貿易辦事處商務投資主任戴博斯（Balazs Szabo）等',
 '田中光表示，台灣智慧農業週暨台灣國際漁業展是台灣農、漁產業界的國際性嘉年華展覽，已邁向第8年，展覽規模、參展的國家與國內外廠商均逐年增加，感謝參展廠商及產學研究機構的努力，將台灣農、漁業優質產品及技術在國際展覽中呈現',
 '訂閱《早安世界》電子報 每天3分鐘掌握10件天下事 請輸入正確的電子信箱格式訂閱感謝您的訂閱',
 '田中光指出，今年有超過15個國家、大約270家廠商參展，其中包括吐瓦魯國大使館、巴布亞紐幾內亞駐台商務代表處、聖文森及格瑞那丁大使館、史瓦帝尼王國大使館、匈牙利貿易辦事處的參展攤位，感謝國際廠商以及外交團的參與，使展覽更具國際性及異國風采',
 '田中光表示，在世界經濟受到COVID-19疫情嚴重衝擊下，這項實體展覽，就像是台灣版的「農、漁業經貿泡泡」，期盼透過大家努力經營，使展覽規模更形壯大',
 '田中光韓駐台代表：台韓半導體業緊密相連使彼此獲益印尼舉辦國慶酒會 駐台代表盼深化與台經貿關係']

## All-in-one function: Find related paragraphs

In [69]:
import re

# split paragraghes in text
def cut_paragraph(text):
    result = re.split(r'[。|！|!|？|?]', text)
    result = list(filter(None, result))
    return result

# Find out all paragraphs where multiple keywords occur.
def get_same_para(df_query, key, cond, k = 30):
    same_para=[]
    for text in df_query.content:
        #print(text)
        paragraphs = cut_paragraph(text)
        for para in paragraphs:
            para += "。"
            if cond=='and':
                if all([re.search(kw, para) for kw in key]):
                    same_para.append(para)
            elif cond=='or':
                if any([re.search(kw, para) for kw in key]):
                    same_para.append(para)
    return same_para[0:k]

In [73]:
key = ['智慧','台灣']
cond ='and'
cate ='全部'
weeks = 2

In [74]:
df_query = filter_dataFrame_fullText(key, cond, cate, weeks)

In [75]:
len(df_query)

9

In [76]:
get_same_para(df_query, key, 'and', k=5)

['外交部政次田中光今天出席2022年「台灣國際漁業展」與「台灣智慧農業週」聯合開幕儀式，他表示，今年有超過15國約270家廠商參展，展覽規模、參展國家與國內外廠商均逐年增加。',
 '田中光表示，台灣智慧農業週暨台灣國際漁業展是台灣農、漁產業界的國際性嘉年華展覽，已邁向第8年，展覽規模、參展的國家與國內外廠商均逐年增加，感謝參展廠商及產學研究機構的努力，將台灣農、漁業優質產品及技術在國際展覽中呈現。',
 '鄭炳元駐台使節瘋爬山 加拿大、韓國代表同登玉山頂台韓觀光交流會議登場  為重啟旅遊準備田中光台灣國際漁業展、智慧農業週開幕 逾15國參展印尼舉辦國慶酒會 駐台代表盼深化與台經貿關係。',
 '吳釗燮說，台灣致力於建構更好的新創環境，在製造業、人工智慧、再生能源及生物科技等產業都極具創造力，台灣也願意貢獻所長，為全球實踐2030年永續發展議程提出創見。',
 '台灣運彩xT1聯盟今天舉行新賽季啟動記者會，各隊總經理除暢談新賽季目標，也大談季外補強，而其中特攻則是各方焦點，除今年洋將鎖定克羅埃西亞與立陶宛的歐陸球員，就連本土「智慧型」中鋒代表曾文鼎，也在特攻鎖定的名單中。']

## All-in-one function: An alternative way for advanced users: using re.seach()

In [79]:
import re

In [80]:
# Find out all paragraphs where multiple keywords occur.
def get_same_para(df_query, key, cond, k = 30):
    same_para=[]
    for text in df_query.content:
        #print(text)
        paragraphs = cut_paragraph(text)
        for para in paragraphs:
            para += "。"
            if cond=='and':
                if all([re.search(kw, para) for kw in key]):
                    same_para.append(para)
            elif cond=='or':
                if any([re.search(kw, para) for kw in key]):
                    same_para.append(para)
    return same_para[0:k]


# views.py in Django website

To save memory, we just import df from the other app as follows.
from app_user_keyword.views import df

In [ ]:
from django.shortcuts import render
from django.views.decorators.csrf import csrf_exempt
from django.http import JsonResponse

from datetime import datetime, timedelta
import pandas as pd
import math
import re
from collections import Counter

# load data method 1
# def load_df_data_v1():
#     # global variable
#     global  df
#     df = pd.read_csv('app_user_keyword_association/dataset/preprocessed_news_218posts.csv', sep='|')

# (3) df can be import from app_user_keyword
# To save memory, we just import df from the other app as follows.
# from app_user_keyword.views import df

# load data method 2
# (4) df can be import from app_user_keyword
import app_user_keyword.views as userkeyword_views
def load_df_data():
    # import and use df from app_user_keyword 
    global df # global variable
    df = userkeyword_views.df

load_df_data()


# For the key association analysis
def home(request):
    return render(request, 'app_user_keyword_association/home.html')

# df_query should be global
@csrf_exempt
def api_get_userkey_associate(request):

    userkey = request.POST.get('userkey')
    cate = request.POST['cate'] # This is an alternative way to get POST data.
    cond = request.POST.get('cond')
    weeks = int(request.POST.get('weeks'))
    key = userkey.split()

    #global  df_query # global variable

    df_query = filter_dataFrame_fullText(key, cond, cate,weeks)
    print(key)
    print(len(df_query))

    if len(df_query) != 0:  # df_query is not empty
        newslinks = get_title_link_topk(df_query, k = 25)
        related_words, clouddata = get_related_words(df_query, key)
        same_paragraph = get_same_para(df_query, key, cond, k = 30) # multiple keywords

    else:
        newslinks = []
        related_words = []
        same_paragraph = []
        clouddata = []

    response = {
        'newslinks': newslinks,
        'related_words': related_words,
        'same_paragraph': same_paragraph,
        'clouddata':clouddata,
    }
    return JsonResponse(response)


def filter_dataFrame_fullText(key, cond, cate,weeks):

    # end date: the date of the last record of news
    end_date = df.date.max()
    print('latest date for dataset:', end_date)

    # start date
    start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')

    # filtering
    if (cate == "全部") & (cond == 'and'):
        df_query = df[(df.date >= start_date) & (df.date <= end_date) & df.content.apply(
            lambda row: all((qk in row) for qk in key))]
    elif (cate == "全部") & (cond == 'or'):
        queryKey = '|'.join(key)
        df_query = df[(df['date'] >= start_date) & (df['date'] <= end_date) & df.content.str.contains(queryKey)]
    elif (cond == 'and'):
        df_query = df[(df.category == cate) & (df.date >= start_date) & (df.date <= end_date) & df.content.apply(
            lambda row: all((qk in row) for qk in key))]
    elif (cond == 'or'):
        queryKey = '|'.join(key)
        df_query = df[(df.category == cate) & (df['date'] >= start_date) & (df['date'] <= end_date) & df[
            'content'].str.contains(queryKey)]

    return df_query


# get titles and links from k pieces of news 
def get_title_link_topk(df_query, k = 25):
    items = []
    for i in range( len(df_query[0: k]) ): # show only 10 news
        category = df_query.iloc[i]['category']
        title = df_query.iloc[i]['title']
        link = df_query.iloc[i]['link']
        photo_link = df_query.iloc[i]['photo_link']
        # if photo_link value is NaN, replace it with empty string 
        if pd.isna(photo_link):
            photo_link=''
        
        item_info = {
            'category': category, 
            'title': title, 
            'link': link, 
            'photo_link': photo_link
        }

        items.append(item_info)
    return items 

# Get related keywords by counting the top keywords of each news.
# Notice:  do not name function as  "get_related_keys",
# because this name is used in Django
def get_related_words(df_query, key):

    all_pairs = {}
    for idx in range(len(df_query)):
        row = df_query.iloc[idx].top_key_freq # column name may be top_keys_freq
        pairs = dict(eval(row))

        # Get related keywords by counting the top keywords of each news.
        for pair in pairs.items():
            w, f = pair
            if w in all_pairs:
                all_pairs[w] += f
            else:
                all_pairs[w] = f
    counter = Counter(all_pairs)

    wf_pairs = counter.most_common(30)

    # cloud chart data
    # the minimum and maximum frequency of top words
    min_ = wf_pairs[-1][1]  # the last line is smaller
    max_ = wf_pairs[0][1]
    # text size based on the value of word frequency for drawing cloud chart
    textSizeMin = 20
    textSizeMax = 120

    clouddata = [{'text': w, 'size': int(textSizeMin + (f - min_) / (max_ - min_) * (textSizeMax - textSizeMin))}
                 for w, f in wf_pairs]

    return   wf_pairs, clouddata 

# split paragraphs
def cut_paragraph(text):
    result = re.split('[。|！|\!|？|\?]', text)
    result = list(filter(None, result))
    return result

# Find out all paragraphs where multiple keywords occur.
def get_same_para(df_query, key, cond, k=30):
    same_para=[]
    for text in df_query.content:
        #print(text)
        paragraphs = cut_paragraph(text)
        for para in paragraphs:
            para += "。"
            if cond=='and':
                if all([re.search(kw, para) for kw in key]):
                    same_para.append(para)
            elif cond=='or':
                if any([re.search(kw, para) for kw in key]):
                    same_para.append(para)
    return same_para[0:k]

    
print("app_user_keyword_association -- 全文檢索與關聯新聞分析載入成功!")

# For reference